## Multi-Agent Supervisor

Based on the example here: 

https://langchain-ai.github.io/langgraph/tutorials/multi_agent/agent_supervisor/#create-agent-supervisor



In [20]:
from typing import Literal
from typing_extensions import TypedDict
import sys

from langchain_anthropic import ChatAnthropic
from langgraph.graph import MessagesState, END
from langgraph.types import Command
from langchain_openai import ChatOpenAI

members = ["researcher", "coder"]
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = members + ["FINISH"]

system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    f" following workers: {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)


class Router(TypedDict):
    """Worker to route to next. If no workers needed, route to FINISH."""

    next: Literal[*options]


llm = ChatOpenAI(model_name="gpt-4o-mini")


class State(MessagesState):
    next: str


def supervisor_node(state: State) -> Command[Literal[*members, "__end__"]]:
    print('Starting the supervisor node with state:',state)
    messages = [
        {"role": "system", "content": system_prompt},
    ] + state["messages"]
    
    response = llm.with_structured_output(Router).invoke(messages)
    print('The response from supervisor llm call is ', response)
    
    goto = response["next"]
    if goto == "FINISH":
        goto = END

    return Command(goto=goto, update={"next": goto})

In [21]:
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent
from ungoliant.tools import polygon_tools as pt

research_agent = create_react_agent(
    llm, tools=[pt.close_price_tool], state_modifier="You are a researcher. DO NOT do any math."
)


def research_node(state: State) -> Command[Literal["supervisor"]]:
    result = research_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="researcher")
            ]
        },
        goto="supervisor",
    )


# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION, WHICH CAN BE UNSAFE WHEN NOT SANDBOXED
code_agent = create_react_agent(llm, tools=[pt.close_price_tool])


def code_node(state: State) -> Command[Literal["supervisor"]]:
    result = code_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="coder")
            ]
        },
        goto="supervisor",
    )


builder = StateGraph(State)
builder.add_edge(START, "supervisor")
builder.add_node("supervisor", supervisor_node)
builder.add_node("researcher", research_node)
builder.add_node("coder", code_node)
graph = builder.compile()

In [22]:
for s in graph.stream(
    {"messages": [("user", "What is the share price range of Apple on 1/14 of 2025?")]}, subgraphs=True
):
    #print(s)
    #print("----")
    print('')

Starting the supervisor node with state: {'messages': [HumanMessage(content='What is the share price range of Apple on 1/14 of 2025?', additional_kwargs={}, response_metadata={}, id='fa3119c8-d2d5-46b1-9d38-1f38042b8b13')]}
The response from supervisor llm call is  {'next': 'researcher'}





Starting the supervisor node with state: {'messages': [HumanMessage(content='What is the share price range of Apple on 1/14 of 2025?', additional_kwargs={}, response_metadata={}, id='fa3119c8-d2d5-46b1-9d38-1f38042b8b13'), HumanMessage(content='The share price of Apple (AAPL) on January 14, 2025, was $233.28. If you need information on price ranges for other dates or data, feel free to ask!', additional_kwargs={}, response_metadata={}, name='researcher', id='05f8c271-6280-48dc-94a0-8da64b87ddb1')], 'next': 'researcher'}
The response from supervisor llm call is  {'next': 'FINISH'}

